# First things first:

## Import all the libraries that will help to clean the database

In [1]:
from functools import reduce
import numpy as np
import pandas as pd
import re

## Import the CSV file 

In [2]:
df = pd.read_csv("/home/carolina/Desktop/IRONHACK/Project files/attacks.csv", encoding="latin-1")

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Start to explore the data


In [4]:
# Rows x Columns

df.shape

(25723, 24)

In [5]:
# Columns name

df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
# Columns that are hidden on the df above:

df[['Age', 'Injury', 'Fatal (Y/N)', 'Time']].head()

,Age,Injury,Fatal (Y/N),Time
0,57,"No injury to occupant, outrigger canoe and pad...",N,18h00
1,11,Minor injury to left thigh,N,14h00 -15h00
2,48,Injury to left lower leg from surfboard skeg,N,07h45
3,NaN,Minor injury to lower leg,N,NaN
4,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN


From this first investigation we can highlight some observations:
- The columns "Unnamed: 22" and "Unnamed: 23" seems to don't have any information. 
- The columns "Case Number 1." and "Case Number 2." looks like duplicated from the column Case Number.
- Some columns such as "pdf", "href formula", "href", "original order" might won't be usefull for the analysis.

If these obervarions are correct we can erase these columns, but before deleting any data we need to make sure.

# First Clean action: Remove duplicates

Before start to manipulate the data, let's check if we have duplicated information.

In [7]:
df.drop_duplicates()


display(df.head())
print(df.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


(25723, 24)


There aren't duplicates on the whole database, but these doesn't mean that we don't have duplicated columns. We will check these information later. 


# Second Action: Clean null values


## Delete columns with null values

Check if the column "Unnamed: 22" and "Unnamed: 23" only have null values.

In [8]:
print(df['Unnamed: 22'].notnull().sum())

1


In [9]:
df['Unnamed: 23'].notnull().sum()

2

As we can see both columns don't have valuable data, so we can delete them.

In [10]:
df = df.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

#df.head()

## Clean null rows

In order to clean the rows, first we need to transpose index and columns. 

Then we will check those "columns" that are fully empty. Their sum of isnull will be 22, because this is the old number of columns that now are index.

After identify these empty columns we will drop them from the dataFrame. 

When we get back to the clean dataFrame, where the index and columns are back to normal, we can see that acctually we have deleted the null rows.

In [11]:
null_row = df.T.isnull().sum().values
null_row_index_22 = df.T.isnull().sum()[df.T.isnull().sum() == 22].index
df = df.drop([e for e in null_row_index_22])


print(df.shape)

(8703, 22)


# Third action: Clean some columns

Let's check those columns that are not usefull for our analysis and delete them.

## Check duplicated columns

Compare the columns "Case Number", "Case Number.1" and "Case Number.2".

1º Do an analysis apart from the dataFrame.

2º Data wrangling - Change all the values of null to 0.

3º See if they are equal("true") or have diffent values("false").

4º With the result we saw that ""Case Number.1" only differ on 20 values with "Case Number.2", which we can say that they are particully the same. Now, when compare to "Case Number" they are 2.000 difent between them.

5º Checking the different rows(those that have "false"). We can see that actually most of then are 0, the rest it is pretty much the same. This means we can delete the duplicate columns and mantain just the first one. 

6º Drop the columns on the dataFrame, not on the apart version we created for this analysis. 

In [12]:
c_df = df[['Case Number','Case Number.1', 'Case Number.2']]

c_df['Case Number'].fillna(0, inplace = True) 
c_df['Case Number.1'].fillna(0, inplace = True) 
c_df['Case Number.2'].fillna(0, inplace = True)

c_df['check'] = c_df['Case Number'] == c_df['Case Number.1']
c_df['check2'] = c_df['Case Number.1'] == c_df['Case Number.2']

#print(c_df["check"].value_counts())
#print(c_df["check2"].value_counts())

decision = c_df[c_df['check'] == False][['Case Number','Case Number.1', 'Case Number.2']]
#decision.head()

/usr/lib/python3/dist-packages/pandas/core/generic.py:5436: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Before delete the columns 'Case Number.1' and 'Case Number.2', let's check if we have a null value in the 'Case Number', so we can replace. 

In [13]:
print(df['Case Number'].isnull().sum())
print(df[df['Case Number'].isnull()][['Case Number', 'Case Number.1']])

1
     Case Number Case Number.1
5488         NaN  1905.09.06.R


In [14]:
df['Case Number'].fillna("1905.09.06.R", inplace = True)

Now, we can drop the following columns because we have all the info on the 'Case Number'.

In [15]:
df = df.drop(columns=['Case Number.1', 'Case Number.2'])

#df.head()

## Delete columns we won't need for analysis


These are columns that are too descritive or that have information that won't be usefull to create any analysis, so we can delete in order to have the database cleaner. 

In [16]:
to_drop = ['Location', 
            'Name',
            'Injury', 
            'Investigator or Source', 
            'pdf', 
            'href formula', 
            'href', 
            'original order']

df.drop(columns=to_drop, inplace=True)

In [17]:
# Now we only have 12 columns and the dataframe already looks much cleaner

df.shape

(8703, 12)

# Four action: Changing the Index

Change the Case Number to index.

In [18]:
df = df.set_index('Case Number')
df.head()

,Date,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Time,Species
Case Number,,,,,,,,,,,
2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,Paddling,F,57,N,18h00,White shark
2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,Standing,F,11,N,14h00 -15h00,NaN
2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,Surfing,M,48,N,07h45,NaN
2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,M,NaN,N,NaN,2 m shark
2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,Free diving,M,NaN,N,NaN,"Tiger shark, 3m"


# Fifth action: Clean by string matching

Now we will edit the data, so we can group by on the analysis phase.

## Type: Only Unprovoked, Provoked and Not available

In [19]:
df.Type.value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [20]:
df["Type"] = df.Type.fillna("Not Available")

In [21]:
dict_type = {'Unprovoked': ['B', 'S'],
            'Not Available': ['I', 'Q']}
            
for k,v in dict_type.items():
    for e in v:
        df.loc[df['Type'].str.startswith(e),'Type'] = k

In [22]:
df['Type'].value_counts()

Unprovoked       5175
Not Available    2954
Provoked          574
Name: Type, dtype: int64

## Activity - This will be one of the main columns for our analysis

In [23]:
df.Activity.value_counts().head(10)

Surfing         971
Swimming        869
Fishing         431
Spearfishing    333
Bathing         162
Wading          149
Diving          127
Standing         99
Snorkeling       89
Scuba diving     76
Name: Activity, dtype: int64

In [24]:
df["Activity"] = df.Activity.fillna("Not Available")

In [25]:
dict_act = {'Surfing':['urf', 'oard'],
            'Fishing':['ish','Spear','Clamm'],
            'Swimming': ['ath', 'loa', 'wim', 'Wad', 'Jump', 'Play', 'Sculli'],
            'Diving': ['div', 'Div', 'Snor'],
            'Kayaking': ['anoe', 'Row', 'Kay'],
            'Treading water': ['alk', 'Dang', 'tanding'],
            'Sailing': ['oat', 'hip'],
            'Fell into the water': ['Sea', 'Fell'],
            'Interacting with sharks': ['hark'],
            'Others': ['uicide', 'urder', 'addle', 'drill', 'eine', 'SUP', 'plashing', 'Escap']}
            
for k,v in dict_act.items():
    for e in v:
        df.loc[df['Activity'].str.contains(e),'Activity'] = k

All those activity that have less than 4 value_counts() can be deleted from our analysis.

In [26]:
counts_Activity = df.groupby("Activity")["Activity"].transform(len)

mask = (counts_Activity > 5)

#df[mask]

In [27]:
df = df[mask]


Activity clean! This is super important because this is one of the columns that we will built our analysin.

In [28]:
df.Activity.value_counts()

Not Available              2945
Surfing                    1562
Swimming                   1517
Fishing                    1192
Diving                      604
Treading water              181
Sailing                     142
Interacting with sharks     109
Kayaking                     82
Others                       44
Fell into the water          39
Name: Activity, dtype: int64

## Sex: M/F

In [29]:
df["Sex "].value_counts()

M      4887
F       618
N         2
M         2
lli       1
Name: Sex , dtype: int64

In [30]:
df["Sex "] = df["Sex "].fillna("Not Available")

In [31]:
dict_sex = {'M':['N', 'M'],
            'Not Available':['.','l']}

for k,v in dict_sex.items():
    for e in v:
        df.loc[df['Sex '].str.startswith(e),'Sex '] = k

In [32]:
#Sex is clean now!:

df["Sex "].value_counts()

M                7798
F                 618
Not Available       1
Name: Sex , dtype: int64

## Fatal: Y/N - Very important for our analysis

In [33]:
df["Fatal (Y/N)"].value_counts()

N          4169
Y          1271
UNKNOWN      68
 N            6
M             1
N             1
2017          1
y             1
Name: Fatal (Y/N), dtype: int64

In [34]:
df["Fatal (Y/N)"] = df["Fatal (Y/N)"].fillna("Not Available")

In [35]:
dict_fat = {'N':['N', 'M', '2', ' '],
            'Not Available':['U'],
            'Y': ['y']}

for k,v in dict_fat.items():
    for e in v:
        df.loc[df["Fatal (Y/N)"].str.startswith(e),"Fatal (Y/N)"] = k

In [36]:
df["Fatal (Y/N)"].value_counts()

N                7077
Y                1272
Not Available      68
Name: Fatal (Y/N), dtype: int64

## Country - Another essential  column for the analysis

Will be difficult to clean all, but we can focus on the countries with most shark attack.

In [37]:
df["Country"].value_counts().head(10)

USA                 2156
AUSTRALIA           1279
SOUTH AFRICA         565
NEW ZEALAND          125
PAPUA NEW GUINEA     123
BRAZIL               106
BAHAMAS              105
MEXICO                86
ITALY                 68
FIJI                  60
Name: Country, dtype: int64

In [38]:
df["Country"] = df["Country"].fillna("Not Available")

In [39]:
dict_count = {'USA':['USA'],
              'AUSTRALIA': ['AUST'],
              'AFRICA': ['AFRICA'],
              'GUINEA': ['NEW GUINEA'],
              'PHILIPPINES': ['PHILIP'],
              'MEXICO': ['MEX'],
              'ITALY': ['ITAL']}

for k,v in dict_count.items():
    for e in v:
        df.loc[df["Country"].str.contains(e),"Country"] = k

All those Countries that have less than 40 shark attack can be deleted from our analysis.

In [40]:
counts_COUNTRY = df.groupby("Country")["Country"].transform(len)

Mask = (counts_COUNTRY > 40)

#df[Mask]

In [41]:
df = df[Mask]

df["Country"].value_counts()

Not Available    2448
USA              2156
AUSTRALIA        1279
AFRICA            567
GUINEA            136
NEW ZEALAND       125
BRAZIL            106
BAHAMAS           105
MEXICO             87
ITALY              69
FIJI               60
REUNION            58
PHILIPPINES        55
NEW CALEDONIA      52
SPAIN              44
MOZAMBIQUE         42
Name: Country, dtype: int64

## Species

Again, will be difficult to clean all these data because is too descritive, but we can clean up the top ones.

In [42]:
df['Species '].value_counts().head()

White shark                                           141
Shark involvement prior to death was not confirmed     79
Invalid                                                77
Shark involvement not confirmed                        77
Tiger shark                                            57
Name: Species , dtype: int64

In [43]:
df['Species '] = df['Species '].fillna("Not Available")

In [44]:
dict_spe = {'White Shark':['hite'],
            'Tiger Shar': ['iger'],
            'Bull Shark': ['ull'],
            'Wobbegong Shark': ['obbe'],
            'Blue Shark': ['lue'],
            'Not Available':['nvalid','uestiona','confirm', 'nvolvement', 'dentif', 'shark']}

for k,v in dict_spe.items():
    for e in v:
        df.loc[df['Species '].str.contains(e),'Species '] = k

## Age:

In [45]:
df["Age"].value_counts().head()

17    140
18    132
15    124
19    124
16    122
Name: Age, dtype: int64

Using REGEX to find any two digits at the beginning of the string.

In [46]:
regex = r'^(\d{2})' 

extr = df['Age'].str.extract(r'^(\d{2})', expand=False)

extr.head()

Case Number
2018.06.18       11
2018.06.09       48
2018.06.08      NaN
2018.06.04      NaN
2018.06.03.b    NaN
Name: Age, dtype: object

In [47]:
df["Age"] = df["Age"].fillna("Not Available")

In [48]:
df['Age'] = pd.to_numeric(extr)

In [49]:
df["Age"].value_counts().head()

17.0    142
18.0    133
20.0    126
19.0    124
15.0    124
Name: Age, dtype: int64

Now, our database already looks much cleaner, but we still can shape it more.

# Sixth action: Categorization

Let's create a new column called Decada that we might use on the analysis. We will build based on the column Year.

In [50]:
df["Decada"] = pd.cut(df.Year,
                                bins=[1949,1959,1969,1979,1989,1999,2009,2019], 
                                labels=range(1950,2020,10))

In [51]:
df["Decada"].value_counts()

2010    915
2000    900
1990    504
1960    474
1980    385
1950    323
1970    273
Name: Decada, dtype: int64

# Last Clean Actions:

## Remove the columns we won't use

Now that we saw the data, we know better those columns that we can delete, because we won't use on the analysis.

In [52]:
df = df.drop(columns=['Date','Area','Time'])


## Drop "Not Available" in the column Country

The information Country is essential for our analysis, so if we don't have this information we can disregard from the database.

In [53]:
df = df.drop(df[df['Country']=="Not Available"].index)

In [54]:
df.shape

(4941, 9)

## Remove all "NaN" from the column Decada

If Decada is NaN it means that this information os either from before 1949 or because the data Year was not fulfill corrected. So, we can drop these data because is useless.

We can use the formula below straight on the database, because the only column that have "NaN" is Decada.

In [55]:
df = df.replace(-1, np.NaN).dropna()

In [56]:
df.shape

(2517, 9)

## Check Again with we have duplicated values

Now that we already done cleaning the database we can see if we have any duplicated row and in case yes, we can delete it.

In [57]:
df.drop_duplicates()

display(df.head())
print(df.shape)

,Year,Type,Country,Activity,Sex,Age,Fatal (Y/N),Species,Decada
Case Number,,,,,,,,,
2018.06.18,2018.0,Unprovoked,USA,Treading water,F,11.0,N,Not Available,2010
2018.06.09,2018.0,Not Available,USA,Surfing,M,48.0,N,Not Available,2010
2018.06.03.a,2018.0,Unprovoked,BRAZIL,Swimming,M,18.0,Y,Tiger Shar,2010
2018.05.27,2018.0,Unprovoked,USA,Fishing,M,52.0,N,Not Available,2010
2018.05.26.b,2018.0,Unprovoked,USA,Treading water,M,15.0,N,Bull Shark,2010


(2517, 9)


## Data Base clean!!!

Started: 25.723 X 24

Finished: 2.517 X 9

## Export the new database to a CSV

In [58]:
df.to_csv(r"/home/carolina/Desktop/IRONHACK/Project files/clean_shark_attacks.csv", index = False)